# Подбор гипер параметров

В модели встроена поддержка optuna для подбора гипер параметров моделей.

In [1]:
import pandas as pd
df = pd.read_csv("data/ml_ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=df,
    columns_names={
        "user_id": "userId",
        "item_id": "movieId",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

In [5]:
from replay.splitters import UserSplitter

splitter = UserSplitter(
    item_test_size=1,
    user_test_size=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=False,
    seed=1234
)

train, test = splitter.split(log)

# Пример на KNN

Если необходимо затюнить конкретную модель, необходимо создать инстанс модели и воспользоваться методом `optimize`.

Из обязательных параметров — `train` и `test`.

Границы перебора задаются с помощью словаря `param_grid` вида `{param: [low, high]}`. 

Можно не передавать сетку, тогда будет использоваться предопределенная для этой модели в атрибуте `_search_space`. 
Посмотрим на него, чтобы понять, что можно перебирать для данной модели и какие границы нам показались разумными.

In [6]:
from replay.models import KNN
model = KNN()
model._search_space

/Users/darel/python/sponge-bob-magic/.new_env2/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


{'num_neighbours': {'type': 'int', 'args': [1, 100]},
 'shrink': {'type': 'int', 'args': [0, 100]}}

In [9]:
model.optimize(train, test, budget=2, param_grid={'num_neighbours': [10, 20]})

[I 2021-03-03 21:07:21,155] A new study created in memory with name: no-name-61e26c7b-dcd7-4432-a43b-2e19673b43a6
03-Mar-21 21:07:21, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
03-Mar-21 21:07:21, replay, DEBUG: Начало обучения KNN
DEBUG:replay:Начало обучения KNN
03-Mar-21 21:07:21, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
03-Mar-21 21:07:22, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
03-Mar-21 21:07:22, replay, DEBUG: Начало предикта KNN
DEBUG:replay:Начало предикта KNN
03-Mar-21 21:07:25, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
03-Mar-21 21:08:32, replay, DEBUG: NDCG=0.03
DEBUG:replay:NDCG=0.03
[I 2021-03-03 21:08:32,934] Trial 0 finished with value: 0.03217300449861628 and parameters: {'num_neighbours': 14}. Best is trial 0 with value: 0.03217300449861628.
03-Mar-21 21:08:32, re

{'num_neighbours': 14}